## 스마트 정수장 알고리즘 - 약품 공정 - 전처리

In [ ]:
import numpy as np
import pandas as pd
import datetime
import scipy
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

if platform.system() == "Darwin":  #
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="NanumGothic")

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",  # ttf 파일이 저장되어 있는 경로
    name="NanumGothic",
)  # 원하는 폰트 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가

plt.rcParams.update({"font.size": 18, "font.family": "NanumGothic"})  # 폰트 설정

plt.rcParams["axes.unicode_minus"] = False

---
### 데이터 전처리

In [ ]:
file = "SN_2020.csv"
load_df = pd.read_csv(file)
load_df = load_df.set_index("시간")
load_df.index = pd.DatetimeIndex(load_df.index)
load_df.head(5)

In [ ]:
df = load_df[
    ["3단계 원수 유입 유량", "3단계 1계열 응집제 주입량", "3단계 1계열 응집제 주입률"]
]

In [ ]:
df.loc[:, "3단계 1계열 응집제 주입률 계산값"] = (
    (df["3단계 1계열 응집제 주입량"] * 1e-3 * 60)
    / (df["3단계 원수 유입 유량"] / 2)
    * 1000
).copy()
df

---
### 2.2. 결측치/이상치 처리

> 실제 데이터에는 수많은 결측치와 이상치들이 존재한다. </br>
> 결측치/이상치가 얼마나 중요하게 작용하는 데이터인지 파악하고, </br>
> 이를 제거하거나 대체해주어야 학습이 아름답게 될 수 있다. </br>
> 결측치/이상치를 제거하는 방법에는 dropna </br>
> 결측치/이상치를 대체하는 방법에는 bfill, ffill, linear interpolation, extrapolation, polynomial interpolation 등이 있다.

In [ ]:
cols = [
    "원수 탁도",
    "원수 pH",
    "원수 알칼리도",
    "원수 전기전도도",
    "원수 수온",
    "3단계 원수 유입 유량",
    "3단계 1계열 응집제 주입률",
    "3단계 침전지 탁도",
]
raw_df = load_df[cols]
raw_df.head(5)

In [ ]:
raw_df.describe()

In [ ]:
mean = raw_df.mean()
mean

In [ ]:
std = raw_df.std()
std

#### 2.2.1. 결측치 탐색

In [ ]:
raw_df.isna().sum()

In [ ]:
raw_df[raw_df["원수 탁도"].isna()]

#### 2.2.2. 이상치 탐색

In [ ]:
col = "원수 알칼리도"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=col)
plt.axhline(mean[col], c="r", label="평균")
plt.axhspan(
    mean[col] - std[col],
    mean[col] + std[col],
    color="grey",
    alpha=0.2,
    label="1 표준편차 구역",
)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
con = raw_df["원수 알칼리도"] > 1e3
raw_df[con]

#### 2.2.3. 결측치/이상치 처리

##### 2.2.3.1. dropna

In [ ]:
df = raw_df.copy()

In [ ]:
df.shape

In [ ]:
df[df["원수 탁도"].isna()].describe()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
con = df["원수 알칼리도"] > 1e4
df = df[~con]

In [ ]:
df.isna().sum()

#### Question: 이렇게 데이터를 삭제해도 될까요? 분석하는데 문제가 없을까요?

##### 2.2.3.2. bfill, ffill

In [ ]:
df = raw_df.copy()

In [ ]:
df = df.bfill().ffill()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

##### 2.2.3.3. linear interpolation

In [ ]:
df = raw_df.copy()

In [ ]:
col = "원수 수온"
fig = plt.figure(figsize=(16, 5))
plt.plot(df.index, df[col], label=col)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
tmp = df["원수 수온"].dropna()
x = tmp.index.to_series().apply(lambda x: x.value * 1e-9 / 60 / 60)
y = tmp.to_numpy()
f1 = scipy.interpolate.interp1d(x, y)
newx = df.index.to_series().apply(lambda x: x.value * 1e-9 / 60 / 60)
newy = f1(newx)
df["원수 수온"] = newy

In [ ]:
col = "원수 수온"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label="원수수온(interpolation적용전)")  # label=col,
plt.plot(df.index, df[col] - 3, c="r", alpha=0.5, label="원수수온(interpolation적용후)")
plt.ylabel(col)
plt.grid()
plt.legend()

---
### 2.3. 학습용 데이터 전처리

> 잘못 운영된 데이터를 학습 데이터로 사용하면 모델은 잘못된 운영을 그대로 모방할 것이다. </br>
> 이를 방지하기 위해 잘못 운영되었다고 판단되는 기간은 삭제를 해야한다.

In [ ]:
df = raw_df.copy()

In [ ]:
con = df["3단계 침전지 탁도"] > 2

plt.figure(figsize=(8, 6))
plt.scatter(df["원수 탁도"], df["3단계 침전지 탁도"], s=3)
plt.scatter(df.loc[con, "원수 탁도"], df.loc[con, "3단계 침전지 탁도"], s=3, c="r")
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("3단계 침전지 탁도")

In [ ]:
con = df["3단계 침전지 탁도"] > 2
df = df.loc[~con]
df

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df["원수 탁도"], df["3단계 침전지 탁도"], s=3)
plt.grid()
plt.xlabel("원수 탁도")
plt.ylabel("3단계 침전지 탁도")

---
### 2.4. 새로운 피쳐 생성

> 모델에 필요하지만 수집하지 못하는 데이터는 기존 데이터로 생성하기도 한다. </br>
> 또한, 하나의 값만 필요한데 두 개 이상의 값이 존재하는 경우, 대표값을 사용하기도 한다.

#### 2.4.1. 침전지 체류 시간

In [ ]:
df = raw_df.copy()

In [ ]:
col = "3단계 원수 유입 유량"
fig = plt.figure(figsize=(16, 5))
plt.plot(df.index, df[col], label=col)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
con = df["3단계 원수 유입 유량"].between(5000, 17500)
df.loc[~con, "3단계 원수 유입 유량"] = np.nan
df = df.ffill().bfill()

In [ ]:
sedimentation_tank_volume = 4.5 * 80 * 18 * 8
df["3단계 침전지 체류시간"] = sedimentation_tank_volume / df["3단계 원수 유입 유량"]

In [ ]:
col = "3단계 원수 유입 유량"
fig = plt.figure(figsize=(16, 5))
plt.plot(raw_df.index, raw_df[col], label=f"{col} (정제 전)")
plt.plot(df.index, df[col], label=f"{col} (정제 후)", c="r")
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
col = "3단계 침전지 체류시간"
fig = plt.figure(figsize=(16, 5))
plt.plot(df.index, df[col], label=f"{col}")
plt.ylabel(col)
plt.grid()
plt.legend()

#### 2.4.2. 대표값

In [ ]:
df = load_df[["4단계 침전지 탁도 1", "4단계 침전지 탁도 2"]]

In [ ]:
df

In [ ]:
df.loc[:, "4단계 침전지 탁도"] = df.mean(axis=1).copy()

In [ ]:
col = "4단계 침전지 탁도"
col1 = "4단계 침전지 탁도 1"
col2 = "4단계 침전지 탁도 2"
st = df.sample(1).index.item()
en = st + pd.Timedelta("1d")
tmp = df[df.index.to_series().between(st, en)]
fig = plt.figure(figsize=(16, 5))
plt.plot(tmp.index, tmp[col1], label=col1, c="b", alpha=0.5)
plt.plot(tmp.index, tmp[col2], label=col2, c="g", alpha=0.5)
plt.plot(tmp.index, tmp[col], label=col, c="r", alpha=1)
plt.ylim(0.2, 0.75)
plt.ylabel(col)
plt.grid()
plt.legend()

---
### 2.5. 리샘플

> 1분 단위의 데이터는 데이터의 양이 너무 많아 학습이 오래 걸릴 수 있다. </br>
> 또한 1분 단위의 데이터는 큰 오차를 가지고 있어 학습을 저해시킬 수 있다. </br>
> 데이터의 품질을 해치지 않는 선에서의 샘플링은 시간과 학습 품질을 높이는 데 도움이 된다.

In [ ]:
df = raw_df.copy()

In [ ]:
col = "원수 pH"
st = df.sample(1).index.item()
en = st + pd.Timedelta("3h")
tmp = df[df.index.to_series().between(st, en)]
fig = plt.figure(figsize=(16, 5))
plt.plot(tmp.index, tmp[col], label=col)
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
df = df.resample("30T").mean().copy()

In [ ]:
col = "원수 pH"
st = df.sample(1).index.item()
en = st + pd.Timedelta("1d")
tmp1 = raw_df[raw_df.index.to_series().between(st, en)]
tmp2 = df[df.index.to_series().between(st, en)]
fig = plt.figure(figsize=(16, 5))
plt.plot(tmp1.index, tmp1[col], label=f"{col} (정제 전)")
plt.plot(tmp2.index, tmp2[col], label=f"{col} (정제 후)", c="r")
plt.ylabel(col)
plt.grid()
plt.legend()

In [ ]:
df